# 分析Prosper数据——EDA

## 目录
* [项目综述](#项目综述)
* [分析过程](#分析过程)
 * [了解数据内容](#了解数据内容)

### 项目综述
项目数据源是Prosper的数据。Prosper是美国一家不同于传统借贷的P2P借贷服务的公司，服务内容是贷款人贷款和借款人获取投资收益，而公司收益是从贷方和借方收取服务费。本次项目是站在投资人的角度，投资的收益是从借款人如约还款来实现，利用EDA的思路分析投资价值——因此需要分析借款人的借款信息和还款信息，期望能够证实P2P借贷方式存在投资价值。

In [1]:
library(ggplot2)
library(RColorBrewer)
library(psych)
library(dplyr)
library(tidyr)


Attaching package: ‘psych’

The following objects are masked from ‘package:ggplot2’:

    %+%, alpha


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
# load data
rawprosper <- read.csv("prosperLoanData.csv", sep=",", stringsAsFactor=FALSE)

In [3]:
dim(rawprosper)

[1] 113937     81

In [6]:
print(sort(names(rawprosper)))

 [1] "AmountDelinquent"                    "AvailableBankcardCredit"            
 [3] "BankcardUtilization"                 "BorrowerAPR"                        
 [5] "BorrowerRate"                        "BorrowerState"                      
 [7] "ClosedDate"                          "CreditGrade"                        
 [9] "CreditScoreRangeLower"               "CreditScoreRangeUpper"              
[11] "CurrentCreditLines"                  "CurrentDelinquencies"               
[13] "CurrentlyInGroup"                    "DateCreditPulled"                   
[15] "DebtToIncomeRatio"                   "DelinquenciesLast7Years"            
[17] "EmploymentStatus"                    "EmploymentStatusDuration"           
[19] "EstimatedEffectiveYield"             "EstimatedLoss"                      
[21] "EstimatedReturn"                     "FirstRecordedCreditLine"            
[23] "GroupKey"                            "IncomeRange"                        
[25] "IncomeVerifiable"     

### 了解数据内容
该数据集的观测值为113937个，具有81个变量。变量中包括了借款人的信息：借款人类似ID信息(MemberKey)，借款人借款开始日期(LoanOriginationDate)，借款人所在州(BorrowerState)，收入范围(IncomeRange)，债务收入比(DebtToIncomeRatio)，职业(Occupation)，借款人工作期间(EmploymentStatusDuration)，借款人工作状态(EmploymentStatus)，借款总额度(LoanOriginalAmount)，还款策略(Term)，借款后进过月数(LoanMonthsSinceOrigination)，交易数量(TotalTrades)，未发生延期交易的比例(TradesNeverDelinquent..percentage.)，账户还款的次数()；另一方面，承担借款额的投资人人数(Investor)，投资款来自朋友的投资人数(InvestmentFromFriendsCount)，来自朋友投资的投资额(InvestmentFromFriendsAmount)。
以上也是本次分析的参考的变量，主要是考虑从了解借款人的收入、债务信息的侧写；另一方面也需要分析投资人在借款人中借款额度中影响，从两个方面去了解借款人的借款合理性、还款能力。以期待能了解Prosper借款用户的特点，分析其投资价值。